In [16]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as  tradeapi
import time

import sys
sys.path.insert(0, '../')
import pandas as pd
from datetime import timedelta

In [17]:
from config import key_id, secret_key
from cross_signal import *

In [18]:
#The url at the end is what makes a paper trading bot
api = tradeapi.REST(key_id, secret_key)

In [25]:
account = api.get_account()
account

Account({   'account_blocked': False,
    'account_number': '010320JEWA',
    'buying_power': '37149.395',
    'cash': '1515.8325',
    'created_at': '2020-03-20T17:46:19.948568Z',
    'currency': 'USD',
    'daytrade_count': 2,
    'daytrading_buying_power': '567.262',
    'equity': '36415.9625',
    'id': '176ffd14-7e0e-4957-9ad5-98306425fa04',
    'initial_margin': '17841.265',
    'last_equity': '34179.89',
    'last_maintenance_margin': '32374.49',
    'long_market_value': '34900.13',
    'maintenance_margin': '34900.13',
    'multiplier': '4',
    'pattern_day_trader': True,
    'portfolio_value': '36415.9625',
    'regt_buying_power': '37149.395',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '31110.19',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [26]:
love=True
from IPython.display import clear_output
while love==True:
    clear_output(wait=True)
    lastquote=api.polygon.last_quote('GME')
    lastprice=lastquote.bidprice
    print(lastquote)
    time.sleep(1)

Quote({   'askexchange': 11,
    'askprice': 320,
    'asksize': 9,
    'bidexchange': 11,
    'bidprice': 318.25,
    'bidsize': 1,
    'timestamp': 1611956425673})


KeyboardInterrupt: 

In [13]:
currenttimestamp


Timestamp('2021-01-29 09:35:06.305169382-0500', tz='pytz.FixedOffset(-300)')

In [11]:
lastquote

Quote({   'askexchange': 8,
    'askprice': 354.35,
    'asksize': 56,
    'bidexchange': 8,
    'bidprice': 352.12,
    'bidsize': 5,
    'timestamp': 1611930761867})

In [14]:
api.polygon.last_quote('GME').timestamp

Timestamp('2021-01-29 09:32:41.867000-0500', tz='America/New_York')

In [10]:
returned = api.submit_order('GME',int(6),"sell","trailing_stop","gtc",trail_price=20)
returned.id

'75948202-122c-4f3e-877c-ddf31d521747'

In [ ]:
api.

In [6]:
assetlist=api.list_positions()
for asset in assetlist:
    print(asset.unrealized_intraday_plpc)

0.0190689848569826
0.010793215692993
0.2975268970801879


In [12]:
quoten=api.polygon.last_quote('TQQQ')
sellprice=quoten.askprice-.04

In [ ]:
api.polygon.last_quote('NIO')

In [10]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdatetime=datetime.datetime.today()
weekdate=(startdatetime-datetime.timedelta(days=14)).strftime("%Y-%m-%d")
monthdate=(startdatetime-datetime.timedelta(days=40)).strftime("%Y-%m-%d")

In [11]:
startdate

'2020-11-23'

In [11]:
from pandas import DataFrame
import talib.abstract as tab

In [13]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=4, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=4, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=4, price='low')
        
        dataframe['dema_close'] = tab.DEMA(dataframe, timeperiod=4, price='close')
        dataframe['sema_high'] = tab.EMA(dataframe, timeperiod=10, price='high')
        dataframe['sema_close'] = tab.EMA(dataframe, timeperiod=10, price='close')
        dataframe['sema_low'] = tab.EMA(dataframe, timeperiod=10, price='low')
        
        dataframe['ssema_close'] = tab.EMA(dataframe, timeperiod=35, price='close')
        dataframe['sssema_close'] = tab.EMA(dataframe, timeperiod=40, price='close')
        dataframe['ssssema_close'] = tab.EMA(dataframe, timeperiod=45, price='close')
        dataframe['chaikosc'] = ta.ADOSC(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'))
        
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        dataframe['obv'] = ta.OBV(dataframe['close'], np.asarray(dataframe['volume'], dtype='float'))
        
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['longminusdi'] = tab.MINUS_DI(dataframe, timeperiod=28)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['longminusdi'] = tab.MINUS_DI(dataframe, timeperiod=35)
        dataframe['longplusdi'] = tab.PLUS_DI(dataframe, timeperiod=35)
        dataframe['splusdi'] = tab.PLUS_DI(dataframe, timeperiod=7)
        dataframe['sma']=tab.SMA(dataframe, timeperiod=10, price='close')
        dataframe['vwap']=dataframe['sma']*dataframe['volume']
        
        dataframe['emaminussdi'] = ta.MINUS_DI(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        dataframe['emaplusdi'] = ta.PLUS_DI(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        dataframe['emaadx'] = ta.ADX(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        
        dataframe['upperband'], dataframe['midband'], dataframe['lowerband'] = ta.BBANDS(dataframe['close'], timeperiod=5, nbdevup=1, nbdevdn=1, matype=0)
        
        dataframe['macd'],dataframe['macdsig'],dataframe['hist']= ta.MACD(dataframe['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14, price='close')
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['shortadx'] = tab.ADX(dataframe, timeperiod=10)
        dataframe['srsi'] = tab.RSI(dataframe, timeperiod=3, price='close')
        dataframe["upperband"], dataframe["middleband"], dataframe["lowerband"] = ta.BBANDS(dataframe['close'],  nbdevup=1.5, nbdevdn=1.5)
        
        dataframe['ema_ultosc'] = ta.ULTOSC(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        dataframe['ema_rsi'] = ta.RSI(dataframe['ema_close'], timeperiod=3)
        dataframe['fema_rsi'] = ta.RSI(dataframe['ema_close'], timeperiod=2)
        dataframe['sema_rsi'] = ta.RSI(dataframe['ema_close'], timeperiod=6)
        
        
        dataframe['beta'] = tab.BETA(dataframe)
        
        dataframe['macd1'],dataframe['macdsig1'],dataframe['hist1']= ta.MACD(dataframe['close'], fastperiod=8, slowperiod=10, signalperiod=7)
        
        
        dataframe['macd_cross1'] =crossed_above(dataframe['macd1'],dataframe['macdsig1'])
        dataframe['macd_cross'] =crossed_above(dataframe['macd'],dataframe['macdsig'])
        dataframe['adxcross1'] =crossed_above(dataframe['adx'],17)
        dataframe['adxcross2'] =crossed_above(dataframe['adx'],27)
        dataframe['adxcross3'] =crossed_above(dataframe['adx'],35)
        
        # required for graphing

        return dataframe

In [13]:
import statistics as sts
def quotelister(quotes):
        #exponential moving averages
        asksizelist=[]
        bidsizelist=[]
        askpricelist=[]
        bidpricelist=[]
        
        
        for quote in quotes:
            if quote.ask_size<50:
                if quote.bid_size<50:
                    asksizelist.append(quote.ask_size)
                    bidsizelist.append(quote.bid_size)
                    bidpricelist.append(quote.bid_size*quote.bid_price)
                    askpricelist.append(quote.ask_size*quote.ask_price)

        # required for graphing
        asksize=sum(asksizelist)
        bidsize=sum(bidsizelist)
        avgaskprice=round(sum(askpricelist)/asksize,2)
        avgbidprice=round(sum(bidpricelist)/bidsize,2)

        return asksize,bidsize,avgbidprice,avgaskprice,minprice

In [14]:
allticks=api.polygon.all_tickers()

In [15]:
api.polygon.snapshot('NUGT').ticker['todaysChangePerc']

-11.889

In [16]:
symli=[]
for ticks in allticks:
    if ticks.prevDay['v'] >50000:
        symli.append(ticks.ticker)

In [24]:
symbol ='NIO'
#returned_data=api.polygon.historic_agg_v2('GOOG', 1, 'minute', _from='2020-01-10', to='2020-01-20').df
#returned_data =api.polygon.historic_agg_v2(symbol, 4, 'minute',_from='2020-07-21', to='2020-07-21').df
returned_data =api.polygon.historic_agg_v2(symbol, 5, 'minute',_from=startdate, to=startdate).df#
#returned_data=api.polygon.historic_agg_v2('ARKG', 4, 'minute', _from='2020-09-03', to='2020-09-03').df
returned_data2=api.polygon.historic_agg_v2('NIO', 1, 'minute', _from='2020-11-23', to='2020-11-23').df

quotes=api.polygon.historic_quotes_v2(symbol,date=startdate,reverse=True,limit=600)
# Calculated trading indicators
data4=populateindicators(returned_data2)
#data5=populateindicators(returned_data2)
import dtale
dtale.show(data4, ignore_duplicate=True)

Executing shutdown due to inactivity...
Executing shutdown due to inactivity...


2020-11-23 12:01:14,323 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...
Executing shutdown...


2020-11-23 12:01:18,337 - INFO     - Executing shutdown...


In [23]:
data4.iloc[-1]['close']> data4.iloc[-1]['open']

False

In [47]:
data4.iloc[-1].name+datetime.timedelta(minutes=3)<api.get_clock().timestamp

True

In [ ]:
cols=pd.Series(df.columns)
for dup in df.columns.get_duplicates(): 
    cols[df.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 
                                     else dup 
                                     for d_idx in range(df.columns.get_loc(dup).sum())]
                                    )
df.columns=cols

In [25]:
dtale.show(result, ignore_duplicate=True)

Exception: data contains duplicated column names: close, high, low, open, volume

Executing shutdown due to inactivity...


2020-09-05 12:12:27,261 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2020-09-05 12:12:31,272 - INFO     - Executing shutdown...


In [41]:
symbol ='TQQQ'
#returned_data=api.polygon.historic_agg_v2('GOOG', 1, 'minute', _from='2020-01-10', to='2020-01-20').df
#returned_data =api.polygon.historic_agg_v2(symbol, 4, 'minute',_from='2020-07-21', to='2020-07-21').df
#returned_data =api.polygon.historic_agg_v2(symbol, 4, 'minute',_from=startdate, to=startdate).df#
returned_data=api.polygon.historic_agg_v2('SPY', 4, 'minute', _from='2020-09-03', to='2020-09-03').df
#returned_data=api.polygon.historic_agg_v2('XOM', 1, 'minute', _from='2020-05-11', to='2020-05-13').df

quotes=api.polygon.historic_quotes_v2(symbol,date=startdate,reverse=True,limit=600)
# Calculated trading indicators
data4=populateindicators(returned_data)

import dtale
dtale.show(data4, ignore_duplicate=True)

Executing shutdown due to inactivity...
Executing shutdown due to inactivity...


2020-09-04 18:01:31,004 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...
Executing shutdown...


2020-09-04 18:01:35,075 - INFO     - Executing shutdown...


In [5]:
quotes=api.polygon.historic_quotes_v2('GME',date=startdate,reverse=True)

NameError: name 'startdate' is not defined

In [24]:
dtale.show(quotes.df, ignore_duplicate=True)

Executing shutdown due to inactivity...


2020-09-04 10:49:26,492 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2020-09-04 10:49:30,513 - INFO     - Executing shutdown...


In [20]:
symbol ='SE'

returned_data2 =api.polygon.historic_agg_v2(symbol, 4, 'minute',_from=startdate, to=startdate).df#

data2=populateindicators(returned_data2)

Executing shutdown due to inactivity...


2020-08-31 12:08:53,827 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2020-08-31 12:08:57,843 - INFO     - Executing shutdown...


In [14]:
love=True
from IPython.display import clear_output

In [ ]:
love=True
from IPython.display import clear_output
while love==True:
    clear_output(wait=True)
    lastquote=api.polygon.last_quote('GME')

    print(lastquote)
    time.sleep(1)
    

Quote({   'askexchange': 11,
    'askprice': 265,
    'asksize': 12,
    'bidexchange': 11,
    'bidprice': 264,
    'bidsize': 1,
    'timestamp': 1611870570662})


In [32]:
returned_data.plot(y='close')

NameError: name 'returned_data' is not defined

In [33]:
asksizelist=[]
bidsizelist=[]
timestamplist=[]

In [34]:
api.polygon.last_quote('AMD')

Quote({   'askexchange': 8,
    'askprice': 85.54,
    'asksize': 3,
    'bidexchange': 12,
    'bidprice': 85.53,
    'bidsize': 1,
    'timestamp': 1598659196087})

In [35]:
import statistics as sts

In [36]:
trades=quotes=api.polygon.historic_trades_v2('GNUS',date=startdate,reverse=True,limit=600).df

In [37]:
asksizelist=[]
bidsizelist=[]
timestamplist=[]
askpricelist=[]
bidpricelist=[]

In [38]:
for quote in quotes:
    asksizelist.append(quote.ask_size)
    bidsizelist.append(quote.bid_size)
    bidpricelist.append(quote.bid_price)
    askpricelist.append(quote.ask_price)
    timestamplist.append(quote.sip_timestamp)

AttributeError: 'str' object has no attribute 'ask_size'

In [39]:
quotes[-1].sip_timestamp

KeyError: -1

In [40]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


FileNotFoundError: [Errno 2] No such file or directory: 'SP500_2020Q2SYM.txt'

In [41]:
quotedf=quotedf.reset_index()

NameError: name 'quotedf' is not defined

In [45]:
symbol ='TSLA'
#returned_data=api.polygon.historic_agg_v2('GOOG', 1, 'minute', _from='2020-01-10', to='2020-01-20').df
#returned_data =api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-27', to='2020-05-04').df
returned_data =api.polygon.historic_agg_v2(symbol, 4, 'minute',_from=startdate, to=startdate).df#
#returned_data=api.polygon.historic_agg_v2('UAL', 1, 'minute', _from='2020-05-13', to='2020-05-13').df
#returned_data=api.polygon.historic_agg_v2('XOM', 1, 'minute', _from='2020-05-11', to='2020-05-13').df

# Calculated trading indicators
data1=populateindicators(returned_data)

import dtale
dtale.show(data1, ignore_duplicate=True)

In [44]:
symbol ='TQQQ'
#returned_data=api.polygon.historic_agg_v2('GOOG', 1, 'minute', _from='2020-01-10', to='2020-01-20').df
#returned_data =api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-27', to='2020-05-04').df
returned_data =api.polygon.historic_agg_v2(symbol, 4, 'minute',_from=startdate, to=startdate).df#
returned_data2 =api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
#returned_data=api.polygon.historic_agg_v2('UAL', 1, 'minute', _from='2020-05-13', to='2020-05-13').df
#returned_data=api.polygon.historic_agg_v2('XOM', 1, 'minute', _from='2020-05-11', to='2020-05-13').df

quotes=api.polygon.historic_quotes_v2(symbol,date=startdate,reverse=True,limit=600)
# Calculated trading indicators
data2=populateindicators(returned_data)
data22=populateindicators(returned_data2)
import dtale
dtale.show(data2, ignore_duplicate=True)
dtale.show(data22, ignore_duplicate=True)

Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\

2020-08-28 22:18:02,700 - ERROR    - Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeff

Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\

2020-08-28 22:18:02,736 - ERROR    - Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeff

Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\

...

2020-08-28 22:18:13,208 - ERROR    - Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeff

Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\

2020-08-28 22:18:14,526 - ERROR    - Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeff

Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\

2020-08-28 22:18:14,527 - ERROR    - Exception on /charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Jeff

In [21]:
symbol ='LABU'
#returned_data=api.polygon.historic_agg_v2('GOOG', 1, 'minute', _from='2020-01-10', to='2020-01-20').df
#returned_data =api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-27', to='2020-05-04').df
returned_data =api.polygon.historic_agg_v2(symbol, 4, 'minute',_from=startdate, to=startdate).df#
#returned_data=api.polygon.historic_agg_v2('UAL', 1, 'minute', _from='2020-05-13', to='2020-05-13').df
#returned_data=api.polygon.historic_agg_v2('XOM', 1, 'minute', _from='2020-05-11', to='2020-05-13').df

quotes=api.polygon.historic_quotes_v2(symbol,date=startdate,reverse=True,limit=600)
# Calculated trading indicators
data2=populateindicators(returned_data)

import dtale
dtale.show(data2, ignore_duplicate=True)

Executing shutdown due to inactivity...
Executing shutdown due to inactivity...


2020-07-16 20:41:46,377 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...
Executing shutdown...


2020-07-16 20:41:50,400 - INFO     - Executing shutdown...


In [14]:
api.polygon.historic_quotes_v2('WOMN',date=startdate,reverse=True,limit=600)

[Quote({   'ask_exchange': 0,
     'ask_price': 0,
     'ask_size': 0,
     'bid_exchange': 8,
     'bid_price': 13.99,
     'bid_size': 7,
     'conditions': [1],
     'indicators': [93],
     'participant_timestamp': 1598644803020324000,
     'sequence_number': 33146752,
     'sip_timestamp': 1598644803020540416,
     'tape': 1}),
 Quote({   'ask_exchange': 0,
     'ask_price': 0,
     'ask_size': 0,
     'bid_exchange': 19,
     'bid_price': 23.14,
     'bid_size': 1,
     'conditions': [1],
     'indicators': [93],
     'participant_timestamp': 1598644803020314000,
     'sequence_number': 33146751,
     'sip_timestamp': 1598644803020530176,
     'tape': 1}),
 Quote({   'ask_exchange': 0,
     'ask_price': 0,
     'ask_size': 0,
     'bid_exchange': 18,
     'bid_price': 23.14,
     'bid_size': 1,
     'conditions': [1],
     'indicators': [93],
     'participant_timestamp': 1598644803020287000,
     'sequence_number': 33146750,
     'sip_timestamp': 1598644803020515072,
     'tape'

In [ ]:
1598634179998283776